In [2]:
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prod_database = { 
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000}, 
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000}, 
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000}, 
} 


def is_product_available(product_name: str)-> bool: 
    """특정 제품의 재고가 있는지 확인한다.

    Args:
        product_name: 제품명
    """    
    if product_name in prod_database: 
        if prod_database[product_name]["재고"] > 0: 
            return True 
    return False 

def get_product_price(product_name: str)-> int:
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database: 
        return prod_database[product_name]["가격"] 
    return None 

def place_order(product_name: str, address: str)-> str: 
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name): 
        prod_database[product_name]["재고"] -= 1 
        return "주문 완료" 
    else: 
        return "재고 부족으로 주문 불가" 


In [4]:
# 모델이 JSON 문자열로 알려준 함수와 실제 함수를 연결하기 위한 매핑 정보
function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}

In [5]:
# tools 필드에 함수 리스트를 세팅
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    tools=function_repoistory.values()
)

In [6]:
chat_session = model.start_chat()
prompt = "갤럭시 S24 판매 중인가요?"
response = chat_session.send_message(prompt)
print(response.candidates[0].content)

parts {
  function_call {
    name: "is_product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "갤럭시 S24"
        }
      }
    }
  }
}
role: "model"



In [7]:
part = response.candidates[0].content.parts[0]
if part.function_call:
    function_call =  part.function_call
    function_name = function_call.name
    function_args = {k: v for k, v in function_call.args.items()}
    print(f"{function_name} args=>: {function_args}")
    function_result = function_repoistory[function_name](**function_args)
    print(f"{function_name} result=>: {function_result}")


is_product_available args=>: {'product_name': '갤럭시 S24'}
is_product_available result=>: True


In [16]:
import google.ai.generativelanguage as glm

part = glm.Part(
    function_response=glm.FunctionResponse(
        name="is_product_available", 
        response={ 
            "content": function_result, 
        }, 
    )
)
print(part)
response = chat_session.send_message(part)
print("-----")
print(response.candidates[0].content)

function_response {
  name: "is_product_available"
  response {
    fields {
      key: "content"
      value {
        bool_value: true
      }
    }
  }
}

-----
parts {
  text: "네, 판매 중입니다. 😊 \n"
}
role: "model"

